<a href="https://colab.research.google.com/github/ritu-kumar/Machine-learning-projects/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Here, data is in textul form. We need to deal with it differently because it is different for computer.
# From given raw data, we pre process it, then train and test split then using logistic regression model, we classifiy either news is fake or true.

In [54]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [55]:
# re is Regular expression, used for searching words in a text or paragraph.
# nltk stands for natural language tool kit, stopwords removes general words that dont contribute to the context, like the, a,etc.
# Tfidfvectoriser converts texts into feature vectors.

In [56]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
print(stopwords.words('english'))
# These are the words from english language that will contribute nothing and will be ignored.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [58]:
news_dataset = pd.read_csv('/content/train.csv')
print(news_dataset)
news_dataset.shape


          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

(20800, 5)

In [59]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [60]:
# To count how many values are missing
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Above data means there are 0 missing values in id, 558 missing values in title and so on. But even with thes sevalues, we can proceed further because we have a large datset.

In [61]:
#Replacing null values with empty string
news_dataset= news_dataset.fillna('')
# Null string can be mentioned by '' i.e. single quotes.

In [62]:
# Merging author name and title 
news_dataset['content'] = news_dataset['author']+ ' ' + news_dataset['title']
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [63]:
# Seperating data and label
x = news_dataset.drop(columns = 'label', axis = 1)
y = news_dataset['label']
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming : It's a process to reduce the word to it's root word.

In [64]:
port_stem = PorterStemmer()

def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ',content)
  # Above line is to remove every thing except lower case a-z and higher cas A-Z. i.e. numbers and symbols are removed. The upper arrow means exclusion.
  stemmed_content = stemmed_content.lower()
  # This step is to convert all the texts in lower case.
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(words) for words in stemmed_content if not words in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [65]:
news_dataset['content'] = news_dataset['content'].apply(stemming)
# Taking content column and applying stemming to it 

In [66]:
print(news_dataset['content'])  
# Now contains only texts in lower case and no numbers.

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [67]:
# Now, whole dataset is not needed. Only the content and label will work for us.
x = news_dataset['content'].values
y = news_dataset['label'].values

In [82]:
print(x.shape)
print(y.shape)

(20800, 17128)
(20800,)


In [69]:
# Computer can't process textuak data. We need to convert it to numbers.
vectoriser = TfidfVectorizer()
# TF means term frequency and idf means inverse document frequency.It counts number of times a word is repeating in a  document.
# Repeatation means a word is important and it assigns a value for that. 
# IDF checks out words which are repeated but dont have special meaning.
vectoriser.fit(x)
x = vectoriser.transform(x)
print(x)
# i.e the whole content is changed in numerical form.

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [83]:
# Splitting the data
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=2)
print(y_train.shape)

(16640,)


In [84]:
model= LogisticRegression()

In [86]:
model.fit(x_train,y_train)


LogisticRegression()

In [88]:
# Evaluation
x_train_prediction = model.predict(x_train)
train_accuracy = accuracy_score(x_train_prediction, y_train)


In [89]:
print("Accuracy score of training data is:", train_accuracy)

Accuracy score of training data is: 0.9865985576923076


In [90]:
# Testing
x_test_prediction = model.predict(x_test)
test_accuracy = accuracy_score(x_test_prediction, y_test)
print("Accuracy of test data is:", test_accuracy)

Accuracy of test data is: 0.9790865384615385


In [92]:
from nltk.util import ProxyDigestAuthHandler
# Making a predictive system
x_new = x_test[1]
prediction = model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print("The news is real.")
else:
  print("The news is fake.")


[0]
The news is real.
